In [39]:
# The code was removed by Watson Studio for sharing.

# Lidl Tweet Stream Using Python and Tweepy

The below code connects to the twitter API and will get a stream of tweets that incluse the Lidl Keyword.

In [50]:
# Import the Tweepy library for using the twitter API and the library for sentiment analysis
!pip install tweepy
!pip install textblob
!python -m textblob.download_corpora
import tweepy
from textblob import TextBlob
import pandas as pd
import re 

# Create all the necessary authentication variables
consumer_key = #Your Key
consumer_secret = #Your Secret
access_token = #Your Token
access_token_secret = #Your  Token Secret

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting the access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

# The search term you want to find
query = "lidl"
# Language code (follows ISO 639-1 standards)
language = "en"
# Number of Tweets to pull
cnt = 100
# Result type (default is mixed)
res = "mixed"

# Calling the user_timeline function with our parameters
results = api.search(q=query, lang=language,count=cnt,result_type=res)

# Function to clean unwanted characters out of a tweets text
def clean_tweet(tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

# Function to get a sentiment score for a tweet
def get_tweet_sentiment(tweet): 

    # create TextBlob object of passed tweet text 
    text = clean_tweet(tweet)
    analysis = TextBlob(text) 
    # set sentiment
    if analysis.sentiment.polarity > 0: 
        return ['positive',analysis.sentiment.polarity]
    elif analysis.sentiment.polarity == 0: 
        return ['neutral',analysis.sentiment.polarity]
    else: 
        return ['negative',analysis.sentiment.polarity]

# Create the data frame
df_tweets = pd.DataFrame(columns=["Tweet ID", "Username", "Tweet Text", "User Profile Location", "Number of Retweets", "Number of Favorites", "Sentiment Score", "Sentiment Category", "Tweet Date"])
i = 0

# Loop over all tweets and get the important data + the sentiment of each tweet
for tweet in results:
    # Get all of the data we want
    Tid = tweet.id
    User = tweet.user.screen_name
    Body = tweet.text
    Location = tweet.user.location
    RTctn = tweet.retweet_count
    Favctn = tweet.favorite_count
    CreDate = tweet.created_at
    
    if Body.startswith("RT"):
        continue
    
    Senti = get_tweet_sentiment(Body)
    SScore = Senti[1]
    Senti = Senti[0]
    TweetRow = [Tid, User, Body, Location, RTctn, Favctn, SScore, Senti, CreDate]
    df_tweets.loc[i] = TweetRow
    i=i+1
    
    
# Read in previous csv file from the project
my_file = project.get_file("Lidl_Tweets.csv")
my_file.seek(0)
df_full_tweets = pd.read_csv(my_file)

# Append the new tweets to the old file
df_full_tweets = df_full_tweets.append(df_tweets)

# Remove Duplicates
df_full_tweets = df_full_tweets.drop_duplicates(subset ="Tweet ID")

# Save the updated tweets file with the new tweets
project.save_data("Lidl_Tweets.csv", df_full_tweets.to_csv(index=False), overwrite=True) 

[nltk_data] Downloading package brown to /home/dsxuser/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dsxuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


{'file_name': 'Lidl_Tweets.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'twittersentimentanalysis-donotdelete-pr-ydy3u38p5q0hxs',
 'asset_id': 'bb4054b3-54a7-4a87-a8e7-4e5a9e8ba7fb'}